In [46]:
import os
import openai
from functools import reduce
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.retrievers import ArxivRetriever
from langchain.retrievers import WikipediaRetriever
#from langchain.embeddings import 
from youtube_transcript_api import YouTubeTranscriptApi

In [47]:
# YouTube Video Subtitles Retrieval
videoID = 'iR2O2GPbB0E&ab'
transcript = YouTubeTranscriptApi.get_transcript(videoID)
transcript = [thisObject["text"].replace("\n", " ") for thisObject in transcript]
transcript = reduce(lambda u, v: u + " " + v, transcript)

print(transcript[:101])

SPEAKER 1: Unless you've been living under a rock, you've probably heard that AI is getting very good


In [48]:
# Credentials
os.environ["OPENAI_API_KEY"] = "76c1a2c33c9a4f4b8bffa3d3c8b51ebb"

In [ ]:
def pipeline(videoID, searchLimit):
    # YouTube Video Subtitles Retrieval
    transcript = YouTubeTranscriptApi.get_transcript(videoID)
    transcript = [thisObject["text"].replace("\n", " ") for thisObject in transcript]
    transcript = reduce(lambda u, v: u + " " + v, transcript)
    
    # Get Summarization
    LLM = ChatOpenAI(temperature=0.0)
    summaryPrompt = ChatPromptTemplate.from_template(
        "Summarize the following text as much as possible:\n\n{text}"
    )
    summaryChain = LLMChain(
        llm=LLM,
        prompt=summaryPrompt
    )
    summary = summaryChain.run(transcript)
    
    # Ask the user its preferable information source
    source = input("Please enter the information source from - Wikipedia, arXiv") # to add more sources
    
    if source == "Wikipedia":
        retriever = WikipediaRetriever(top_k_results=searchLimit)
    else:
        retriever = ArxivRetriever(load_max_docs=searchLimit)
        
    documents = retriever.get_relevant_documents(query=summary)
    
    return documents